In [37]:
########Reading data from Mongo#######
from pyspark.sql import SparkSession
import re
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

spark = SparkSession.builder.master("local[*]").config("spark.mongodb.input.uri","mongodb://localhost:27017/twitter_db.Dummy").config("spark.mongodb.output.uri","mongodb://locahost:27017/twitter_db.Dummy").config("spark.jars.packages","org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").getOrCreate()
df = spark.read.format("mongo").option("uri","mongodb://localhost:27017/twitter_db.Dummy").load()
df.registerTempTable("Twitter")
#df.count()

In [ ]:
sf_1=df.select("text")
words = sf_1.select(explode(split(df.text, "t_end")).alias("word"))
word=df.select(F.regexp_replace('text', r'http\S+', '').alias("text"))
word=word.select(F.regexp_replace('text', '@\w+', '').alias("text"))
word=word.select(F.regexp_replace('text', '#', '').alias("text"))
word=word.select(F.regexp_replace('text', 'RT', '').alias("text"))
word=word.select(F.regexp_replace('text', ':', '').alias("text"))
word.show(truncate=False)

In [99]:
##########Feature Selection############
#import langdetect import detect_

#Selecting required columns

sf=df.select(explode(split(df.text, "t_end")).alias("text"),"created_at",col("user.location").alias("Location"),"retweet_count",col("user.followers_count").alias("User_followers"),col("user.favourites_count").alias("favourites_count"),col("user.verified").alias("Verified User"),"lang")
sf=sf.select(F.regexp_replace('text', r'http\S+', '').alias("Text"),"created_at","Location","retweet_count","favourites_count","Verified User","User_followers","lang")
sf=sf.select(F.regexp_replace('Text', '@\w+', '').alias("text"),"created_at","Location","retweet_count","favourites_count","Verified User","User_followers","lang")
sf=sf.select(F.regexp_replace('text', '#', '').alias("text"),"created_at","Location","retweet_count","favourites_count","Verified User","User_followers","lang")
sf=sf.select(F.regexp_replace('text', 'RT', '').alias("text"),"created_at","Location","retweet_count","favourites_count","Verified User","User_followers","lang")
sf=sf.select(F.regexp_replace('text', ':', '').alias("Text"),from_unixtime(unix_timestamp('created_at', 'EEE MMM d HH:mm:ss z yyyy'),format="yyyy-MM-dd").alias('date'),"Location","User_followers","favourites_count","retweet_count","Verified User")
sf=sf.fillna({"Location": "unknown"})
sf=sf.fillna({"retweet_count": 0})
sf=sf.filter((col("lang") == 'en'))

sf.show()
#af.printSchema()
#af.show()
#sf.withColumn('newDate',f.date_sub("created_at",10)).show()
#df.show(2)

+--------------------+----------+--------------------+--------------+----------------+-------------+-------------+
|                Text|      date|            Location|User_followers|favourites_count|retweet_count|Verified User|
+--------------------+----------+--------------------+--------------+----------------+-------------+-------------+
|  Ajith Starrer V...|2021-01-14|             Twitter|            91|            4154|            0|        false|
|       Powerful edit|2021-01-14|   Aurangabad, India|           803|           12989|            0|        false|
|watch it 👀
pleas...|2021-01-14|    Bengaluru, India|            25|              67|            0|        false|
|Cigarette scene I...|2021-01-14|       Mumbai, India|         10124|            9125|            0|        false|
| Then how can thi...|2021-01-14|             unknown|          1404|           17010|            0|        false|
|  Shah Rukh Khan ...|2021-01-14|              india |          2947|          20

In [53]:
#Counting total tweets
af.where(col("retweet_count").isin({"0"}))

DataFrame[Text: string, date: string, location: string, retweet_count: int, verified: boolean]

In [100]:
############Writing structured data into Mongo###########
sf.write.format("mongo").option("uri","mongodb://localhost:27017/Sample2.Data").mode("append").save()